##  All tables

This code is largely based on the implementation provided by Enge et al. (2021), available at https://osf.io/34ry2/. We are deeply grateful for their dedication to open research.

> Enge, A., Abdel Rahman, R., & Skeide, M. A. (2021). A meta-analysis of fMRI studies of semantic cognition in children. NeuroImage, 241, 118436. https://doi.org/10.1016/j.neuroimage.2021.118436

In [1]:
# Import necessary modules
from os import makedirs, path
from atlasreader import get_statmap_info
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from nilearn import image, plotting, reporting
from scipy import stats
from scipy.stats import pearsonr
import os, fnmatch
from glob import glob
from pathlib import Path
import re
from nibabel import save

# print the current working directory
# print(os.getcwd())

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [2]:
def combined_cluster_table(
    img_files_z=None,
    img_files_ale=None,
    stub_keys=None,
    stub_colname="Analysis",
    cluster_extent=200,
    atlas="aal",
    td_jar=None,
    output_file="",
):
    if img_files_z is None:
        img_files_z = []
    if img_files_ale is None:
        img_files_ale = []
    if stub_keys is None:
        stub_keys = []

    # Create output directory
    output_dir = path.dirname(output_file)
    makedirs(output_dir, exist_ok=True)

    # Create a list of DataFrames with peak and cluster stats for each Z image
    df_tuples = [
        get_statmap_info(
            img_file,
            cluster_extent=cluster_extent,
            atlas=atlas,
            voxel_thresh=0,
        )
        for img_file in img_files_z
    ]

    dfs = [
        pd.DataFrame(
            {
                "Cluster #": df_tuple[0]["cluster_id"],
                "Size (mm3)": df_tuple[0]["volume_mm"],
                "Cluster labels": df_tuple[0][atlas],
                "Mean z": df_tuple[0]["cluster_mean"],
                "Peak z": df_tuple[1]["peak_value"],
                "Peak X": df_tuple[1]["peak_x"],
                "Peak Y": df_tuple[1]["peak_y"],
                "Peak Z": df_tuple[1]["peak_z"],
                "Peak label": df_tuple[1][atlas],
            }
        )
        for df_tuple in df_tuples
    ]

    # Add ALE values if available
    if img_files_ale:
        df_tuples_ale = [
            get_statmap_info(
                img_file,
                cluster_extent=cluster_extent,
                atlas=atlas,
                voxel_thresh=0,
            )
            if img_file
            else (
                pd.DataFrame({"cluster_mean": [float("nan")]}),
                pd.DataFrame({"peak_value": [float("nan")]}),
            )
            for img_file in img_files_ale
        ]

        dfs_ale = [
            pd.DataFrame(
                {
                    "Mean ALE": df_tuple[0]["cluster_mean"],
                    "Peak ALE": df_tuple[1]["peak_value"],
                }
            )
            for df_tuple in df_tuples_ale
        ]

        for df, df_ale in zip(dfs, dfs_ale):
            df.insert(4, column="Mean ALE", value=df_ale["Mean ALE"])
            df.insert(6, column="Peak ALE", value=df_ale["Peak ALE"])

    # Concatenate into one big DataFrame
    df = pd.concat(dfs, keys=stub_keys)

    # Reformat numerical columns
    df["Size (mm3)"] = df["Size (mm3)"].apply(lambda x: "{:,.0f}".format(x))

    cols_int = ["Cluster #", "Peak X", "Peak Y", "Peak Z"]
    
    df[cols_int] = df[cols_int].applymap(
    lambda x: int(x) if pd.notnull(x) else ""
    )

    cols_2f = ["Mean z", "Peak z"]
    df[cols_2f] = df[cols_2f].applymap(lambda x: "{:,.2f}".format(x))

    if img_files_ale:
        cols_3f = ["Mean ALE", "Peak ALE"]
        df[cols_3f] = df[cols_3f].applymap(lambda x: "{:,.3f}".format(x))
        df[cols_3f] = df[cols_3f].replace("nan", "")

    # Add the stub column
    df.index = df.index.set_names(stub_colname, level=0)
    df.reset_index(level=stub_colname, inplace=True)

    mask = df[stub_colname].duplicated()
    df.loc[mask.values, [stub_colname]] = ""

    # Save to CSV
    df.to_csv(output_file, sep="\t", index=False)

    return df


## Tables - ALE results

#### Table 1 - Single ALE Analysis (Self- versus Other- condition), included:

1. Schizophrenia;
2. Healthy Controls Minus Psychiatric Patients;
3. Psychiatric Patients Minus Healthy Controls.

In [3]:
tab1 = combined_cluster_table(
    img_files_z=[
        "../Output/1_ALE/heper_and_hypo_z_size_level_thresh.nii.gz",
        "../Output/1_ALE/control_minus_patient_z_size_level_thresh.nii.gz",
        "../Output/1_ALE/patient_minus_control_z_size_level_thresh.nii.gz",
    ],
    stub_keys=[
        "hyper_and_hypo",
        "Control > Patient",
        "Patient > Control",
    ],
    cluster_extent=10,
    stub_colname="Analysis",
    img_files_ale=[
        "../Output/1_ALE/heper_and_hypo_stat_size_thresh.nii.gz",
        "../Output/1_ALE/control_minus_patient_stat_size_thresh.nii.gz",
        "../Output/1_ALE/patient_minus_control_stat_size_thresh.nii.gz",
    ],
    atlas="aal",
    output_file="../Output/4_Tables/tab_ALE.tsv"
)

display(tab1)

/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,hyper_and_hypo,1,896,33.93% Precuneus_R; 33.93% no_label; 32.14% Ci...,2.36,0.012,2.36,0.011,14,-44,38,Precuneus_R
0,Control > Patient,1,"1,248",34.62% Precuneus_R; 33.33% Cingulate_Mid_R; 32...,3.54,0.011,3.54,0.011,14,-44,38,Precuneus_R


#### Table 2: ALE Contrast Analysis. (Self- versus Other- condition)

In [19]:
# Compute seperate difference maps for controls > patients and patients > controls

img_sub = image.load_img("../Output/2_Contrast/Control_all_minus_patient_z_thresh.nii.gz")
img_control_gt_patient = image.math_img("np.where(img > 0, img, 0)", img=img_sub)
img_patient_gt_control = image.math_img("np.where(img < 0, img * -1, 0)", img=img_sub)
_ = save(img_control_gt_patient, "../Output/2_Contrast/control_greater_patient_z_thresh.nii.gz")
_ = save(img_patient_gt_control,  "../Output/2_Contrast/patient_greater_control_z_thresh.nii.gz")

In [24]:
# Create a combined cluster table for the subtraction analysis
tab2 = combined_cluster_table(
    img_files_z=[
        "../Output/2_Contrast/control_greater_patient_z_thresh.nii.gz",
        "../Output/2_Contrast/patient_greater_control_z_thresh.nii.gz",
    ],
    stub_keys=[
        "control_greater_patient", 
        "patient_greater_control"
    ],
    cluster_extent=10,
    stub_colname="Analysis",
    img_files_ale=[None, None],
    atlas="aal",
    output_file="../Output/4_Tables/tab_contrast.tsv"
)

display(tab2)


/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/Users/ss/miniconda3/lib/python3.12/site-packages/atlasreader/atlasreader.py:496: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  thresh_img = image.threshold_img(stat_img, threshold=voxel_thresh)
/var/folders/yk/78rqxlxn4pz_rsb5_31xvh340000gn/T/ipykernel_31937/3619135046.py:89: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_i

,Analysis,Cluster #,Size (mm3),Cluster labels,Mean z,Mean ALE,Peak z,Peak ALE,Peak X,Peak Y,Peak Z,Peak label
0,control_greater_patient,,nan,NaN,nan,,nan,,,,,NaN
0,patient_greater_control,1,"1,584",93.94% Frontal_Inf_Tri_R,3.42,,3.89,,52,28,16,Frontal_Inf_Tri_R
